# Basic Pathway

In this system we consider that our reactions in the main pathway don't interfere with the environment around them, and vice versa. 

This is our first trial in modeling the propane producing pathway, and it may help us to understand better how the production works in the real world.

## Pathway

<img style="width:50%;height:50%;" src="http://www.biotechnologyforbiofuels.com/content/figures/s13068-015-0231-1-1-l.jpg">

In [4]:
%matplotlib inline
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt

ImportError: libpng15.so.15: cannot open shared object file: No such file or directory

## Reactions:

$$\frac{d[Acetoacetyl-CoA]}{dt} =  k_{AtoB}[Acetyl-CoA]^2 - k_{Hbd}[Acetoacetyl-CoA][NADPH]  $$ 

$$\frac{d[3-hydroxybutyryl-CoA]}{dt} =  k_{Hbd}[Acetoacetyl-CoA][NADPH] - k_{Crt}[3-hydroxybutyryl-CoA][H_2O]  $$

$$\frac{d[Crotonyl-CoA]}{dt} =  k_{Crt}[3-hydroxybutyryl-CoA][H_2O] - k_{Ter}[Crotonyl-CoA][NADH]  $$

$$\frac{d[Butyryl-CoA]}{dt} =  k_{Ter}[Crotonyl-CoA][NADH] - k_{YciA}[Butyryl-CoA][H_2O]  $$ 

$$\frac{d[Butyric-acid]}{dt} =  k_{YciA}[Butyryl-CoA][H_2O] - k_{CAR}[Butyric acid][NADPH][H_2O][ATP] $$

$$\frac{d[Butyraldehyde]}{dt} = k_{CAR}[Butyric acid][NADPH][H_2O][ATP] - k_{ADO}[Butyraldehyde][NADPH]^2[H_+]^2[O_2] $$

$$\frac{d[Propane]}{dt} = k_{ADO}[Butyraldehyde][NADPH]^2[H+]^2[O2] $$

In [ ]:
# Define constants
K_AtoB = 0.47
K_Hbd = 0.05
K_Crt = 0.03
K_Ter = 0.0052
K_YciA = 0.013
K_CAR = 0.362
K_ADO = 1
H_2O = 38.85
ATP = 0.00000223 # 2.23 micro mol/l, is this right?

# A = Acetoacetyl-CoA
# B = Acetyl-CoA
# C = NADPH
# D = 3-hydroxybutyryl-CoA
# E = Crotonyl-CoA
# F = Butyryl-CoA
# G = Butyric-acid
# H = Butyraldehyde
# I = H_+; 2,511e-8 mol/L
# J = O_2; interp1([30,42],[9 6.5],37)/32 is about 0.236 mol/l

# Define system in terms of separated differential equations
def r1(A,B,C):
    return K_AtoB*A - K_Hbd*B*C

def r2(A,C,D):
    return K_Hbd*A*C - K_Crt*D*H_2O

def r3(C,D,E):
    return  K_Crt*D*H_2O - K_Ter*E*C

def r4(E,C,F):
    return K_Ter*E*C - K_YciA*F*H_2O

def r5(F,G,C):
    return K_YciA*F*H_2O - K_CAR*G*C*H_2O*ATP

def r6(G,C,H,I,J):
    return K_CAR*G*C*H_2O*ATP - K_ADO*H*C**2*I**2*J**2

def r7(H,C,I,J):
    return K_ADO*H*C**2*I**2*J**2


# initialize lists containing values
a = []
b = []
c = []
d = []
e = []
f = []
g = []

# Time Constant
T = 2000
# Timestep Constant
dT = 0.01

#iv1, iv2 = initial values, dt = timestep, time = range
def sys(iv1, iv2, iv3, iv4, iv5, iv6, iv7, dt, time):
    # initial values:
    a.append(iv1)
    b.append(iv2)
    c.append(iv3)
    d.append(iv4)
    e.append(iv5)
    f.append(iv6)
    g.append(iv7)
    #z.append(iv3)
    # compute and fill lists
    for i in range(time):
        a.append(a[i] + (r1(a[i],1,1)) * dt)
        b.append(b[i] + (r2(a[i],1,b[i])) * dt)
        c.append(c[i] + (r3(1,b[i],c[i])) * dt)
        d.append(d[i] + (r4(c[i],1,d[i])) * dt)
        e.append(e[i] + (r5(d[i],e[i],1)) * dt)
        f.append(f[i] + (r6(e[i],1,f[i],0.00000002511,0.236)) * dt)
        g.append(g[i] + (r7(f[i],1,0.00000002511,0.236)) * dt)
        #z.append(z[i] + (h(x[i],y[i],z[i])) * dt)
    return a, b, c, d, e, f, g

sys(100, 0, 0, 0, 0, 0, 0,  dT, T)

#plot
fig = plt.figure(figsize=(15,5))
fig.subplots_adjust(wspace = 0.5, hspace = 0.3)
ax1 = fig.add_subplot(1,2,1)
#ax2 = fig.add_subplot(1,2,2)

ax1.plot(a, 'r-', label='Acetoacetyl-CoA')
ax1.plot(b, 'b-', label='3-hydroxybutyryl-CoA')
ax1.plot(c, 'g-', label='Crotonyl-CoA')
ax1.plot(d, 'c-', label='Butyryl-CoA')
ax1.plot(e, 'm-', label='Butyric-acid')
ax1.plot(f, 'y-', label='Butyraldehyde')
ax1.plot(g, 'k-', label='Propane')

ax1.set_title("Dynamics in time")
ax1.set_xlabel("time")
ax1.set_ylabel("Concentrations")
ax1.grid()
ax1.legend(loc='best')

#ax2.plot(x, y, color="blue")
#ax2.set_xlabel("x")
#ax2.set_ylabel("y")  
#ax2.set_title("Phase space")
#ax2.grid()

In [ ]:
# Calculating the derivatives of the concentrations according to time

# Derivatives
a_D = []
b_D = []
c_D = []
d_D = []
e_D = []
f_D = []
g_D = []

# Calculate derivatives using timestep dT, through time T
for i in range(T-1):
    a_D.append(-(a[i+1]-a[i])/dT)
    b_D.append(-(b[i+1]-b[i])/dT) 
    c_D.append(-(c[i+1]-c[i])/dT)
    d_D.append(-(d[i+1]-d[i])/dT)
    e_D.append(-(e[i+1]-e[i])/dT)
    f_D.append(-(f[i+1]-f[i])/dT)
    g_D.append(-(g[i+1]-g[i])/dT)

#plot
fig = plt.figure(figsize=(30,8))
fig.subplots_adjust(wspace = 0.5, hspace = 0.3)
ax2 = fig.add_subplot(1,2,1)

ax2.plot(a_D, 'r-', label='Acetoacetyl-CoA')
ax2.plot(b_D, 'b-', label='3-hydroxybutyryl-CoA')
ax2.plot(c_D, 'g-', label='Crotonyl-CoA')
ax2.plot(d_D, 'c-', label='Butyryl-CoA')
ax2.plot(e_D, 'm-', label='Butyric-acid')
ax2.plot(f_D, 'y-', label='Butyraldehyde')
ax2.plot(g_D, 'k-', label='Propane')

ax2.set_title("Negative derivatives of concentrations in time")
ax2.set_xlabel("time")
ax2.set_ylabel("Negative Derivative of Concentration")
ax2.grid()
ax2.legend()